In [1]:
import math
import random
import re

In [2]:
from jmetal.core.problem import PermutationProblem
from jmetal.core.solution import PermutationSolution

In [3]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm
from jmetal.operator import BinaryTournamentSelection
from jmetal.operator.crossover import PMXCrossover
from jmetal.operator.mutation import PermutationSwapMutation
from jmetal.util.comparator import MultiComparator
from jmetal.util.density_estimator import CrowdingDistance
from jmetal.util.ranking import FastNonDominatedRanking
from jmetal.util.termination_criterion import StoppingByEvaluations

In [4]:
"""
.. module:: TSP
   :platform: Unix, Windows
   :synopsis: Single Objective Traveling Salesman problem

.. moduleauthor:: Antonio Benítez-Hidalgo <antonio.b@uma.es>
"""
class TSP(PermutationProblem):
    """ Class representing TSP Problem. """

    def __init__(self, instance: str = None):
        super(TSP, self).__init__()

        distance_matrix, number_of_cities = self.__read_from_file(instance)

        self.distance_matrix = distance_matrix

        self.obj_directions = [self.MINIMIZE]
        self.number_of_variables = number_of_cities
        self.number_of_objectives = 1
        self.number_of_constraints = 0

    def __read_from_file(self, filename: str):
        """
        This function reads a TSP Problem instance from a file.

        :param filename: File which describes the instance.
        :type filename: str.
        """

        if filename is None:
            raise FileNotFoundError('Filename can not be None')

        with open(filename) as file:
            lines = file.readlines()
            data = [line.lstrip() for line in lines if line != ""]

            dimension = re.compile(r'[^\d]+')

            for item in data:
                if item.startswith('DIMENSION'):
                    dimension = int(dimension.sub('', item))
                    break

            c = [-1.0] * (2 * dimension)

            for item in data:
                if item[0].isdigit():
                    j, city_a, city_b = [x.strip() for x in item.split(' ')]
                    c[2 * (int(j) - 1)] = float(city_a)
                    c[2 * (int(j) - 1) + 1] = float(city_b)

            matrix = [[-1] * dimension for _ in range(dimension)]

            for k in range(dimension):
                matrix[k][k] = 0

                for j in range(k + 1, dimension):
                    dist = math.sqrt((c[k * 2] - c[j * 2]) ** 2 + (c[k * 2 + 1] - c[j * 2 + 1]) ** 2)
                    #dist = round(dist)
                    matrix[k][j] = dist
                    matrix[j][k] = dist

            return matrix, dimension

    def evaluate(self, solution: PermutationSolution) -> PermutationSolution:
        fitness = 0

        for i in range(self.number_of_variables - 1):
            x = solution.variables[i]
            y = solution.variables[i + 1]

            fitness += self.distance_matrix[x][y]

        first_city, last_city = solution.variables[0], solution.variables[-1]
        fitness += self.distance_matrix[first_city][last_city]

        solution.objectives[0] = fitness

        return solution


    def create_solution(self) -> PermutationSolution:
        new_solution = PermutationSolution(number_of_variables=self.number_of_variables,
                                           number_of_objectives=self.number_of_objectives)
        new_solution.variables = random.sample(range(self.number_of_variables), k=self.number_of_variables)

        return new_solution


    @property
    def number_of_cities(self):
        return self.number_of_variables

    def get_name(self):
        return 'Symmetric TSP'


In [5]:
def calc_tsp(file_location,population_size_ga,offspring_population_size_ga,max_evaluation_ga):
    problem = TSP(instance=file_location)

    print('Cities: ', problem.number_of_variables)

    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=population_size_ga,
        offspring_population_size=offspring_population_size_ga,
        mutation=PermutationSwapMutation(1.0 / problem.number_of_variables),
        crossover=PMXCrossover(0.8),
        selection=BinaryTournamentSelection(
            MultiComparator([FastNonDominatedRanking.get_comparator(),
                             CrowdingDistance.get_comparator()])),
        termination_criterion=StoppingByEvaluations(max = max_evaluation_ga)
    )

    algorithm.run()
    result = algorithm.get_result()

    print('Algorithm: {}'.format(algorithm.get_name()))
    print('Problem: {}'.format(problem.get_name()))
    print('Solution: {}'.format(result.variables))
    print('Fitness: {}'.format(result.objectives[0]))
    print('Computing time: {}'.format(algorithm.total_computing_time))
    

In [6]:
djibouti_tsp_dat = 'C:/Users/zshnh/Downloads/dj38.tsp'
qatar_tsp_dat = 'C:/Users/zshnh/Downloads/qa194.tsp'

In [7]:
calc_tsp(djibouti_tsp_dat,100,100,2500000)

Cities:  38
Algorithm: Genetic algorithm
Problem: Symmetric TSP
Solution: [9, 0, 1, 3, 7, 8, 27, 30, 26, 18, 17, 16, 10, 11, 15, 23, 21, 22, 19, 14, 5, 2, 4, 6, 12, 24, 25, 13, 20, 28, 31, 32, 33, 35, 37, 36, 34, 29]
Fitness: 10362.367343403906
Computing time: 636.3592913150787


In [8]:
calc_tsp(qatar_tsp_dat,100,100,2500000)

Cities:  194
Algorithm: Genetic algorithm
Problem: Symmetric TSP
Solution: [168, 163, 153, 118, 91, 72, 50, 60, 66, 99, 140, 138, 136, 125, 113, 75, 71, 74, 102, 117, 151, 152, 149, 155, 133, 129, 97, 112, 141, 182, 186, 167, 169, 179, 184, 177, 180, 176, 174, 131, 124, 64, 62, 73, 80, 78, 94, 157, 164, 187, 183, 150, 111, 53, 31, 9, 11, 39, 37, 30, 27, 23, 10, 13, 22, 25, 56, 67, 28, 21, 26, 8, 2, 1, 3, 70, 81, 139, 126, 110, 89, 88, 105, 104, 106, 115, 132, 165, 170, 154, 130, 147, 134, 142, 161, 178, 189, 171, 148, 145, 143, 146, 119, 109, 82, 95, 100, 98, 93, 103, 144, 173, 172, 156, 108, 86, 79, 58, 61, 101, 120, 127, 123, 122, 116, 114, 83, 63, 32, 20, 17, 38, 33, 55, 51, 48, 49, 54, 41, 45, 43, 59, 87, 92, 76, 18, 14, 29, 34, 40, 42, 46, 57, 36, 44, 96, 107, 128, 135, 121, 85, 84, 19, 0, 5, 15, 12, 16, 24, 35, 7, 6, 4, 68, 90, 77, 69, 65, 52, 47, 137, 162, 160, 175, 181, 193, 185, 158, 166, 159, 192, 188, 191, 190]
Fitness: 25983.051203846648
Computing time: 6942.3977472782135


In [ ]:
if __name__ == '__main__':
    problem = TSP(instance='C:/Users/zshnh/Downloads/dj38.tsp')

    print('Cities: ', problem.number_of_variables)

    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=100,
        offspring_population_size=100,
        mutation=PermutationSwapMutation(1.0 / problem.number_of_variables),
        crossover=PMXCrossover(0.8),
        selection=BinaryTournamentSelection(
            MultiComparator([FastNonDominatedRanking.get_comparator(),
                             CrowdingDistance.get_comparator()])),
        termination_criterion=StoppingByEvaluations(max = 2500000)
    )

    algorithm.run()
    result = algorithm.get_result()

    print('Algorithm: {}'.format(algorithm.get_name()))
    print('Problem: {}'.format(problem.get_name()))
    print('Solution: {}'.format(result.variables))
    print('Fitness: {}'.format(result.objectives[0]))
    print('Computing time: {}'.format(algorithm.total_computing_time))

In [ ]:
if __name__ == '__main__':
    problem = TSP(instance='C:/Users/zshnh/Downloads/dj38.tsp')

    print('Cities: ', problem.number_of_variables)

    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=200,
        offspring_population_size=100,
        mutation=PermutationSwapMutation(1.0 / problem.number_of_variables),
        crossover=PMXCrossover(0.8),
        selection=BinaryTournamentSelection(
            MultiComparator([FastNonDominatedRanking.get_comparator(),
                             CrowdingDistance.get_comparator()])),
        termination_criterion=StoppingByEvaluations(max = 2500000)
    )

    algorithm.run()
    result = algorithm.get_result()

    print('Algorithm: {}'.format(algorithm.get_name()))
    print('Problem: {}'.format(problem.get_name()))
    print('Solution: {}'.format(result.variables))
    print('Fitness: {}'.format(result.objectives[0]))
    print('Computing time: {}'.format(algorithm.total_computing_time))

In [ ]:
if __name__ == '__main__':
    problem = TSP(instance='C:/Users/zshnh/Downloads/qa194.tsp')

    print('Cities: ', problem.number_of_variables)

    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=200,
        offspring_population_size=100,
        mutation=PermutationSwapMutation(1.0 / problem.number_of_variables),
        crossover=PMXCrossover(0.8),
        selection=BinaryTournamentSelection(
            MultiComparator([FastNonDominatedRanking.get_comparator(),
                             CrowdingDistance.get_comparator()])),
        termination_criterion=StoppingByEvaluations(max = 2500000)
    )

    algorithm.run()
    result = algorithm.get_result()

    print('Algorithm: {}'.format(algorithm.get_name()))
    print('Problem: {}'.format(problem.get_name()))
    print('Solution: {}'.format(result.variables))
    print('Fitness: {}'.format(result.objectives[0]))
    print('Computing time: {}'.format(algorithm.total_computing_time))